In [1]:
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.io as pio

#pio.renderers.default = "browser"

In [4]:
%cd ..

/Users/joseangelvelasco/GoogleDrive/thesis/timeseries/iberdrola


In [62]:
df = pd.read_csv("data/iberdrola_demand.csv", sep=";", dtype={'time': str, 'demand': np.int})
df['demand_wh'] = df[' demand_wh'].astype(int)
df['date'] = pd.to_datetime(df['time'], format='%d/%m/%Y %H:%M:%S')
df.drop(['time', ' demand_wh'], axis=1, inplace=True)

In [63]:
df.head()

,demand_wh,date
0,490,2019-10-03 00:00:00
1,315,2019-10-03 01:00:00
2,194,2019-10-03 02:00:00
3,137,2019-10-03 03:00:00
4,124,2019-10-03 04:00:00


In [64]:
# df.rename(columns={' demand_wh': 'demand_wh'}, inplace=True)
df['demand_kwh'] = df['demand_wh'].apply(lambda x: x * 1 / 1000)
df.drop(['demand_wh'], axis=1, inplace=True)
df['energy_cumulated_kwh'] = np.cumsum(df['demand_kwh'])

In [65]:
df.tail()

,date,demand_kwh,energy_cumulated_kwh
306,2019-12-26,0.199,85.328
307,2019-12-26,0.190,85.518
308,2019-12-26,0.197,85.715
309,2019-12-26,0.202,85.917
310,2019-12-26,0.176,86.093


In [67]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['date'],
                         y=df['demand_kwh'],
                         name='Daily energy demand (kWh)')
              )
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.update_layout(title="Hourly Energy Demand",
                  yaxis_title='Hourly Energy Demand (kWh)',
                  xaxis_title='Time')
fig.show()

In [42]:
df_aux = df[df['date']<'2019-10-14']
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_aux['date'],
                         y=df_aux['demand_kwh'],
                         name='Daily energy demand (kWh)')
              )
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.update_layout(title="Hourly Energy Demand",
                  yaxis_title='Hourly Energy Demand (kWh)',
                  xaxis_title='Time')
fig.show()

In [68]:
df_aux = df[df['date']<pd.to_datetime('2019-10-14')]
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_aux['date'],
                         y=df_aux['energy_cumulated_kwh'],
                         name='Energy')
          )
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.update_layout(title="Energy Cumulated",
              yaxis_title='Energy consuption (kWh)',
              xaxis_title='Time (h)')
fig.show()

In [69]:
deltaE = round((df_aux.iloc[-1]['energy_cumulated_kwh'] - df_aux.loc[0]['energy_cumulated_kwh'] ) /len(df_aux) ,3)
print('Consumo medio horario: {} kWh'.format(deltaE))

Consumo medio horario: 0.277 kWh


In [86]:
ts = df.set_index(['date'])[['demand_kwh']].resample('D').sum()
ts.head()

,demand_kwh
date,
2019-10-03,7.597
2019-10-04,7.307
2019-10-05,5.881
2019-10-06,6.413
2019-10-07,7.528


In [90]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ts['2019-10-03':'2019-10-14'].index,
                         y=ts['2019-10-03':'2019-10-14']['demand_kwh'],
                         name='Daily energy demand (kWh)')
              )
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"))
fig.update_layout(title="Daily Energy Demand",
                  yaxis_title='Daily Energy Demand (kWh)',
                  xaxis_title='Days')
fig.show()




In [97]:
start = '2019-10-03'
end = '2019-10-14'

df_aux = df[(df['date']>start) &(df['date']<end)]

ts_aux = ts[(ts.index>start) & (ts.index<end) ]


fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_aux['date'],
               y=df_aux['energy_cumulated_kwh'],
               name="Cumulated Energy Consumption"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=ts_aux.index,
               y=ts_aux['demand_kwh'],
               name="Daily Energy Demand"),
    secondary_y=False,
)
# Set x-axis title
fig.update_xaxes(title_text="Time (h)")

# Set y-axes titles
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"),
                  title_text="Household energy demand",
                  legend=dict(x=0.5, y=1.2))
fig.update_yaxes(title_text="<b>Cumulated Energy Consumption (kWh)</b>",
                 secondary_y=True,
                 color="blue")
fig.update_yaxes(title_text="<b>Daily Energy Demand (kWh)</b>",
                 secondary_y=False,
                 color="red")

fig.show()

In [ ]:
start = '2019-10-03'
end = '2019-10-14'

df_aux = df[(df['date']>start) &(df['date']<end)]

ts_aux = ts[(ts.index>start) & (ts.index<end) ]


fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_aux['date'],
               y=df_aux['energy_cumulated_kwh'],
               name="Cumulated Energy Consumption"),
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(x=ts_aux.index,
               y=ts_aux['demand_kwh'],
               name="Daily Energy Demand"),
    secondary_y=False,
)
# Set x-axis title
fig.update_xaxes(title_text="Time (h)")

# Set y-axes titles
fig.update_layout(font=dict(family="Courier New, monospace", size=18, color="#7f7f7f"),
                  title_text="Household energy demand",
                  legend=dict(x=0.5, y=1.2))
fig.update_yaxes(title_text="<b>Cumulated Energy Consumption (kWh)</b>",
                 secondary_y=True,
                 color="blue")
fig.update_yaxes(title_text="<b>Daily Energy Demand (kWh)</b>",
                 secondary_y=False,
                 color="red")

fig.show()